<a href="https://colab.research.google.com/github/eric19950925/PythonColabTTU/blob/main/PythonTTU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

練習1:直接爬原始碼


In [ ]:
from urllib import request
url = "https://ctee.com.tw/livenews/aj/page/1"
res = request.urlopen(url)
content = res.read().decode('utf-8')
print(content)

練習2:爬指定字串

In [1]:
from bs4 import BeautifulSoup
import requests as rq
r = rq.get(url)
print(r)
soup = BeautifulSoup(r.text,"lxml")
print(soup.find('h1').text)

例外處理(P29)

練習3:新聞文章彙整


In [ ]:
def get_web_page(url):
  resp = rq.get(
      url = url,
      cookies = {'over18': '1'}
  )
  if resp.status_code != 200:
    print('invalid url:',resp.url)
    return None
  else: 
    return resp.text

In [ ]:
def get_articles(dom,date):
  soup = BeautifulSoup(dom,'html5lib')

  paging_div = soup.find('div','btn-group btn-group-paging')
  prev_url = paging_div.find_all('a')[1]['href']

  articles = []
  divs = soup.find_all('div','r-ent')

  for d in divs:
    if d.find('div','date').text.strip() == date:
      push_count = 0
      push_str = d.find('div','nrec').text

      if push_str:
        try:
          push_count = int(push_str)
        except ValueError:
          if push_str == '爆':
              push_count == 99
          elif push_str.startswith('X'):
            push_count == -10

      if d.find('a'):
        href = d.find('a')['href']
        title = d.find('a').text
        author = d.find('div','author').text if d.find('div','author') else ''
        articles.append({
            'title':title,
            'href':href,
            'push_count':push_count,
            'author':author
            })

  return articles,prev_url

In [ ]:
PTT_URL = 'https://www.ptt.cc'
import time
import json

current_page=get_web_page(PTT_URL + '/bbs/Gossiping/index.html')

if current_page:
  articles = []

  today = time.strftime("%m/%d").lstrip('0')

  current_articles,prev_url = get_articles(current_page,today)

  while current_articles:
    articles += current_articles
    current_page = get_web_page(PTT_URL + prev_url)
    current_articles,prev_url = get_articles(current_page,today)

  print('今天有',len(articles),'篇文章')
  threshold = 50
  print('熱門文章(>%d推):'%(threshold))
  for a in articles:
    if int(a['push_count'])>threshold:
      print(a)
  
  with open('gossiping.json','w',encoding='utf-8') as f:
    json.dump(articles, f, indent=2, sort_keys=True, ensure_ascii=False)

今天有 1443 篇文章
熱門文章(>50推):
{'title': '[新聞] 快訊／花蓮4高中生北濱公園戲水！1人遭', 'href': '/bbs/Gossiping/M.1634464064.A.017.html', 'push_count': 80, 'author': 'greenmiracle'}
{'title': '[新聞] 城中城大火男子原稱冒險救弟 坦承將弟反', 'href': '/bbs/Gossiping/M.1634464252.A.BE3.html', 'push_count': 84, 'author': 'ininder30cm'}
{'title': '[新聞] 花蓮健走出現神秘嘉賓！馬英九、金溥聰現', 'href': '/bbs/Gossiping/M.1634464260.A.06A.html', 'push_count': 73, 'author': 'hTCU11'}
{'title': '[新聞] 男子接連撞車造成2傷竟還偷車肇事逃逸  警方追緝中', 'href': '/bbs/Gossiping/M.1634462107.A.1DF.html', 'push_count': 71, 'author': 'goatcheese'}
{'title': '[問卦] 黃賭毒 哪個最難戒?', 'href': '/bbs/Gossiping/M.1634462380.A.06D.html', 'push_count': 55, 'author': 'ANava'}
{'title': 'Re: [新聞] 盪鞦韆3分鐘換人 1百位小孩曝心聲', 'href': '/bbs/Gossiping/M.1634460842.A.D91.html', 'push_count': 79, 'author': 'rich22084'}
{'title': '[問卦] 霍爾的移動城堡 是動產還是不動產？', 'href': '/bbs/Gossiping/M.1634459053.A.E0F.html', 'push_count': 62, 'author': 'jw8878'}
{'title': '[問卦] 男28歲還沒有汽車駕照很奇怪嗎？', 'href': '/bbs/Gossiping/M.1634456945.A.A84.h